In [1]:
import torch

from Modules import LoadingModule
from Modules import Features_encoder
from Modules import quantizationModule
from Modules import wav2vec_transformer
from Modules import ContrastiveLoss

from Modules import TempLibriSpeech

In [2]:
"""

#data loader module init
StandardScalerTransform = LoadingModule.StandardScalerTransform
LargeDataModule = LoadingModule.LargeDataModule("./data/Librispeech", batch_size=16, num_workers=1, transform=StandardScalerTransform)
"""

'\n\n#data loader module init\nStandardScalerTransform = LoadingModule.StandardScalerTransform\nLargeDataModule = LoadingModule.LargeDataModule("./data/Librispeech", batch_size=16, num_workers=1, transform=StandardScalerTransform)\n'

In [3]:
#Temp import dataloader ### rendre compatible PLightning quand on aura le GPU
# en attendant import manuel
from torch.utils.data import DataLoader



dataset = TempLibriSpeech.LibriSpeech(split="train-clean-100", target_length=16000, device='cuda')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [4]:
for i, (audio, text) in enumerate(data_loader):
    print(f"Exemple {i+1}")
    print(f"Audio shape: {audio.shape}")
    print(f"Texte: {text}")
    print("-" * 50)
    if i == 1: 
        break

Exemple 1
Audio shape: torch.Size([16, 16000])
Texte: ('THE FACTOR THAT UNDERLIES ALL THE PERPLEXITIES AND MOST OF THE CONTENTMENT OF MARRIAGE IS ITS UNIQUE DEGREE OF CONCENTRATED INTIMACY HERE THE SUPREME TESTING ALWAYS COMES EACH MEANS SO MUCH TO THE OTHER', "YOUR HONOUR REPLIED THE CORPORAL KNOWS OF TOM'S MISFORTUNES BUT THIS AFFAIR HAS NOTHING TO DO WITH THEM ANY FURTHER THAN THIS", "YES I SUPPOSE IT'S SO WELL SENATOR BALLOON PUT FIFTEEN CENTS WORTH OF STAMPS ON EACH OF THOSE SEVEN HUGE BOXES OF OLD CLOTHES AND SHIPPED THAT TON OF SECOND HAND RUBBISH OLD BOOTS AND PANTALOONS AND WHAT NOT THROUGH THE MAILS AS REGISTERED MATTER", 'I IMAGINED THAT THEY WOULD BE DISGUSTED UNTIL', "BUT PIERCED BY ONE ONLY THE MOTTO BEING SHE ALONE THE HEART WAS MADE OF A SINGLE RUBY AS BIG AS AN OSTRICH'S EGG", 'STAY COUNT HE ADDED YOU WHO MAY BE CALLED THE EMPEROR IF I CLAIM THE TITLE OF KING OF FINANCE HAVE YOU MANY PIECES OF PAPER OF THIS SIZE EACH WORTH A MILLION', 'MISTER WILMINGTON AND MISSUS MUNG

In [5]:
### Model dev ###

In [6]:
import torch
import torch.nn as nn

class Model_W2V(nn.Module):
    def __init__(self, embed_size, num_heads, dropout, forward_expansion, kernel_size, groups, d_model, num_layers, max_relative_position):

        #EAB
        self.batch_size = batch_size
        #self.seq_length = seq_length
        self.embed_size = embed_size
        self.mask_prob = 0.50
        self.mask_length = 10
        self.num_heads = num_heads
        self.dropout = dropout
        self.forward_expansion = forward_expansion
        self.kernel_size = kernel_size
        self.groups = groups
        self.d_model = d_model
        self.num_layers = num_layers

        self.num_codebooks = 2
        self.num_codes = 320
        
        self.code_dim = 256
        self.output_dim = 512
        self.temperature= 0.05

        self.max_relative_position = max_relative_position

        super(Model_W2V, self).__init__()

        

        self.FeaturesEncoder = Features_encoder.FeatureEncoder(input_channels=1, feature_dim=512) #1501 ?
        self.masking = wav2vec_transformer.MaskingWithLearnableEmbedding()
        # d_model, num_heads, dropout, forward_expansion):

        #embed_size, num_heads, dropout, forward_expansion,max_relative_position):
        self.TranformerBlock = wav2vec_transformer.TransformerBlockW(self.embed_size, self.num_heads, self.dropout, self.forward_expansion, self.max_relative_position)   #(self.embed_size, self.num_heads, self.dropout, self.forward_expansion, self.kernel_size, self.groups, self.d_model, self.max_relative_position)
        self.quantization = quantizationModule.QuantizationModule(
            input_dim=512,  # Should match feature_dim from FeatureEncoder
            codebook_size=self.num_codes,
            num_codebooks=self.num_codebooks,
            output_dim=self.output_dim,
            temperature=self.temperature
        )

        # (K , k temp, G codevectorgroup, Vcodevectorpergroup, a 0,05)
        
        self.LossItem = ContrastiveLoss.Wav2vec2Loss(K=100,k=self.temperature, G=self.num_codebooks, V = self.num_codes, a=0.05)
        

    
    def forward(self, x):


       # print("ORIGINAL , ", x.shape)
        x = x.to(next(self.parameters()).device)
        x = x.unsqueeze(1)

        x = self.FeaturesEncoder(x)
        
       #
        
        
        # print("q",x.shape)
        
        quantized_repr, diversity_loss = self.quantization(x)
        
        masked_reps, mask = self.masking(x, self.mask_prob, self.mask_length) #(self, x, mask_prob, mask_length)
        
        contextualized_reps = self.TranformerBlock(masked_reps, masked_reps, masked_reps, mask)
                                                # value, key, query, mask=None
        

        #print("Debug", contextualized_reps.shape, quantized_repr.shape, mask.shape)
        
        
        # context_repr, quantized_repr, perplexity, time_mask_indices):

        
        loss = self.LossItem(contextualized_reps, quantized_repr, diversity_loss, mask)
        
        # print("Context Representation shape:", contextualized_reps.shape)
        # print("Quantized Representation shape:", quantized_repr.shape)
        # print("Mask Indices shape:", masked_reps.shape)
        # mask = torch.tensor(mask)
        # print("Unique Mask Indices values:", mask.unique())

        
   # embed_size, num_heads, dropout, forward_expansion, kernel_size, groups,d_model
        
        return x, contextualized_reps, loss
    

In [7]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm


loss_123 = []

def train_model(model, dataset, epochs, learning_rate, device):

    dataloader = DataLoader(dataset, batch_size=model.batch_size, shuffle=True)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    
    for epoch in range(epochs):
        
        epoch_loss = 0
        total_loss = 0.0

        num_batches = len(dataloader) - 1

        for batch_idx, (inputs, _) in enumerate(tqdm( data_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            
            if batch_idx >= num_batches:
                print("break")
                break  # S'arrêter avant la dernière itération

            optimizer.zero_grad()
            
            _,_, loss = model(inputs)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            total_loss += loss.item()

            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{batch_idx+1}/{num_batches}], Loss: {loss.item():.4f}")
                loss_123.append(loss.item())
                
        avg_loss = total_loss / num_batches
        print(f"Epoch [{epoch+1}/{epochs}] Average Loss: {avg_loss:.4f}")

    return loss_123


In [ ]:
batch_size = 16
seq_length = 151
embed_size = 512
num_heads = 8
dropout = 0.0
forward_expansion = 4
kernel_size = 7
groups = 2
d_model = 512
num_layers = 12

max_relative_position=128
torch.autograd.set_detect_anomaly(True)

device = 'cuda'#torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Model_W2V(embed_size, num_heads, dropout, forward_expansion, kernel_size, groups, embed_size, num_layers, max_relative_position).to(device)


loss_123 = train_model(model, dataset, epochs=7, learning_rate=5e-4, device=device)


Epoch 1/7:   0%|          | 1/1784 [00:03<1:50:40,  3.72s/it]

Epoch [1/7], Step [1/1783], Loss: 5.9177


Epoch 1/7:   6%|▌         | 101/1784 [02:03<32:46,  1.17s/it]

Epoch [1/7], Step [101/1783], Loss: 5.7112


Epoch 1/7:  11%|█▏        | 201/1784 [04:02<32:11,  1.22s/it]

...

Epoch 7/7:  17%|█▋        | 301/1784 [06:06<30:23,  1.23s/it]

Epoch [7/7], Step [301/1783], Loss: 5.7161


Epoch 7/7:  22%|██▏       | 401/1784 [08:07<27:00,  1.17s/it]

Epoch [7/7], Step [401/1783], Loss: 5.7149


Epoch 7/7:  26%|██▋       | 469/1784 [09:29<26:49,  1.22s/it]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(loss_123)

In [ ]:
1

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [27]:
len(dataset)

28539

In [ ]:
model.eval()

# Créez un DataLoader pour le jeu de données de test
dataset = TempLibriSpeech.LibriSpeech(split="test-clean", target_length=480000, device='cuda')
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

all_quantized_reps = []
all_contextualized_reps = []

# Calcul de la perte de reconstruction ou de quantification
total_loss = 0
with torch.no_grad():
    for batch_idx, batch in enumerate(data_loader):
        inputs, _ = batch  # Ici, _ signifie qu'il n'y a pas de labels
        
        inputs = inputs.to('cuda')  # Assurez-vous que les inputs sont sur le bon device

        # Passe avant
        quantized_repr, contextualized_reps, loss = model(inputs)

        # Ajout des représentations à la liste
        all_quantized_reps.append(quantized_repr.cpu().numpy())
        all_contextualized_reps.append(contextualized_reps.cpu().numpy())

        # Accumuler la perte
        total_loss += loss.item()

# Calcul de la moyenne de la perte sur l'ensemble du dataset
average_loss = total_loss / len(data_loader)

# Affichage des résultats
print(f"Average Loss: {average_loss:.4f}")


In [ ]:
average_loss = total_loss / len(data_loader)

# Affichage des résultats
print(f"Average Loss: {average_loss:.4f}")

In [ ]:
inputs

In [16]:
torch.cuda.empty_cache()